In [69]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer
dev = qml.device("default.qubit")
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from pennylane.optimize import NesterovMomentumOptimizer

In [111]:
num_qubits = 3
num_layers = 4
@qml.qnode(dev)
def circuit(x, weights):
    # Angle embedding
    # print("Input:",x)
    qml.templates.AngleEmbedding(x, wires=range(num_qubits))

    # Basic entangler layers
    qml.templates.StronglyEntanglingLayers(weights, wires=range(num_qubits))

    # qml.CNOT(wires=[10,0])
    # Measurement
    return qml.expval(qml.PauliX(0))


In [112]:
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc


In [113]:
def cost(weights, X, Y):
    predictions = circuit(X, weights)
    
    # Sigmoid activation to convert to probabilities
    predictions = 1 / (1 + np.exp(-predictions,requires_grad = True))

    # Calculate BCE loss
    bce_loss = -np.mean(Y * np.log(predictions + 1e-10) + (1 - Y) * np.log(1 - predictions + 1e-10))
    return bce_loss

In [114]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the dataset (adjust the path to your file)
df = pd.read_csv('credit_risk_dataset.csv')

# Display the first few rows


# Check for missing values
print("Missing values in each column:\n", df.isnull().sum())

# Preprocessing: Fill or drop missing values as necessary
# df.fillna(0, inplace=True)

df.dropna(inplace=True)
df = df.reset_index(drop=True)
df = df.select_dtypes(exclude=['object'])
# Identify categorical columns
# categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Initialize LabelEncoder
# label_encoder = LabelEncoder()

# Encode categorical columns
# for col in categorical_cols:
    # df[col] = label_encoder.fit_transform(df[col])

# Define features and target variable
X = df.drop('loan_status', axis=1)  # Replace 'loan_status' with your target variable name
# X = X.drop(['person_emp_length', 'person_age'],axis=1)
X = X[['person_age','person_income','person_emp_length']]
y = df['loan_status']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test= train_test_split(X_test, y_test, test_size=0.5, random_state=42)
# print(X_train.iloc[0])

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
y_train[y_train==0] = -1
y_test[y_test==0] = -1
y_val[y_val==0] = -1
# print("Shapes of the training and test sets:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")


Missing values in each column:
 person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64
X_train: (22910, 3), y_train: (22910,)
X_test: (2864, 3), y_test: (2864,)


In [115]:
np.random.seed(0)
weights_init = np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
# bias_init = np.array(0.0, requires_grad=True)
#
# print("Weights:", weights_init)
# print("Bias: ", bias_init)

In [116]:
circuit_draw = qml.draw(circuit,level="device")(X_train[0], weights_init)  # Visualize with the first training sample
print(circuit_draw)
print(X_train[0])

0: ──RX(-0.43)──Rot(1.76,0.40,0.98)───╭●────╭X──Rot(0.41,0.14,1.45)──╭●─╭X──Rot(0.31,-0.85,-2.55)
1: ──RX(0.59)───Rot(2.24,1.87,-0.98)──╰X─╭●─│───Rot(0.76,0.12,0.44)──│──╰●─╭X────────────────────
2: ──RX(-0.67)──Rot(0.95,-0.15,-0.10)────╰X─╰●──Rot(0.33,1.49,-0.21)─╰X────╰●────────────────────

────────────────────────╭●────╭X──Rot(-0.19,1.53,1.47)──╭●─╭X────┤  <X>
───Rot(0.65,0.86,-0.74)─╰X─╭●─│───Rot(0.15,0.38,-0.89)──│──╰●─╭X─┤     
───Rot(2.27,-1.45,0.05)────╰X─╰●──Rot(-1.98,-0.35,0.16)─╰X────╰●─┤     
[-0.43340234  0.58547569 -0.66894316]


In [117]:
weights = weights_init
# bias = bias_init
opt = NesterovMomentumOptimizer(0.01)
batch_size = 32
costs = []
for it in range(1000):

    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    # batch_index_val = np.random.randint(0, len(X_val), (batch_size,))
    X_batch = X_train[batch_index]
    # print(X_batch)
    Y_batch = y_train[batch_index]

    # X_val_b = X_val[ batch_index_val]
    # Y_val_b = y_val[ batch_index_val]
    
    # print(Y_batch)
    weights= opt.step(cost, weights,  X=X_batch, Y=Y_batch)

    # Compute accuracy
    predictions = [np.sign(circuit(x,  weights)) for x in X_batch]
    # print(predictions)
    # print(Y_batch)
    current_cost = cost(weights,  X_batch, Y_batch)
    costs.append(current_cost)
    acc = accuracy(Y_batch, predictions)
    
    # predictions = [np.sign(circuit(x,  weights)) for x in X_val_b]
    # val_acc =accuracy(Y_val_b, predictions)

    if it%10 == 0:
        print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

Iter:    1 | Cost: 0.7699111 | Accuracy: 0.3750000
Iter:   11 | Cost: 0.3270239 | Accuracy: 0.8125000
Iter:   21 | Cost: 0.1404438 | Accuracy: 0.8125000
Iter:   31 | Cost: 0.2430893 | Accuracy: 0.6562500
Iter:   41 | Cost: 0.0660526 | Accuracy: 0.7500000
Iter:   51 | Cost: 0.0721625 | Accuracy: 0.7187500
Iter:   61 | Cost: -0.0003898 | Accuracy: 0.8125000
Iter:   71 | Cost: 0.1036998 | Accuracy: 0.6875000
Iter:   81 | Cost: 0.0855872 | Accuracy: 0.7187500
Iter:   91 | Cost: 0.0297616 | Accuracy: 0.7500000
Iter:  101 | Cost: -0.0040487 | Accuracy: 0.7500000
Iter:  111 | Cost: -0.0986129 | Accuracy: 0.8125000
Iter:  121 | Cost: -0.2253760 | Accuracy: 0.8750000
Iter:  131 | Cost: -0.1247283 | Accuracy: 0.7812500
Iter:  141 | Cost: -0.0269018 | Accuracy: 0.7500000
Iter:  151 | Cost: -0.1402913 | Accuracy: 0.8437500
Iter:  161 | Cost: -0.0603833 | Accuracy: 0.7500000
Iter:  171 | Cost: -0.1021917 | Accuracy: 0.8125000
Iter:  181 | Cost: -0.1070847 | Accuracy: 0.7812500
Iter:  191 | Cost: -0

In [118]:
predictions = [np.sign(circuit(x,  weights)) for x in X_test]
acc = accuracy(y_test, predictions)

In [119]:
print(acc)

0.7692039106145251


In [120]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

# Step 6: Make predictions
y_pred = model.predict(X_test)

# Step 7: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

Accuracy: 0.78
Classification Report:
               precision    recall  f1-score   support

          -1       0.78      1.00      0.88      2238
           1       0.00      0.00      0.00       626

    accuracy                           0.78      2864
   macro avg       0.39      0.50      0.44      2864
weighted avg       0.61      0.78      0.69      2864



C:\Users\cgamb\.conda\envs\Quantum\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\cgamb\.conda\envs\Quantum\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\cgamb\.conda\envs\Quantum\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [121]:
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred_dt = dt_model.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))

Decision Tree Accuracy: 0.7849162011173184
